<a href="https://colab.research.google.com/github/JeanCarloTS/Machine-Learning-Projects/blob/main/Est%C3%A1gio/ML_Dados_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Bibliotecas**

In [1]:
import pandas as pd                   # manipulação de base de dados
import numpy as np                    # manipulação numérica
import seaborn as sns                 # manipulaçõa gráfica
import matplotlib.pyplot as plt       # manipulação gráfica

# **2. Importação dos dados**

## **2.1 Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Estágio - IMT & NPL Brasil/Notebooks/DataFrame_final_Twitter.csv', error_bad_lines=False)

## **2.2 Local**

In [2]:
df = pd.read_csv(r'C:\Users\jcts1\Desktop\JEAN\GitHub\Machine-Learning-Projects\Estágio\Twitter\Arquivos_e_dados_finais\DataFrame_final_Twitter.csv', error_bad_lines=False)

In [3]:
df

,text,likes_faixa,likes
0,Tremores de terra sacudiram Goma na República ...,-1,7
1,O novo presidente do Equador Guillermo Lasso f...,-1,13
2,Novos dados sobre pesquisadores de Wuhan aumen...,0,39
3,RT A Ericsson disse que pode obter uma fatia ...,-1,0
4,Aeroportos devem ser foco para conter a varian...,0,42
...,...,...,...
16235,Este conteúdo faz parte da NexoEDUnoEnem noss...,-1,26
16236,AcessoLivre Ao longo do mês de dezembro o Ne...,-1,19
16237,Este conteúdo faz parte da NexoEDUnoEnem noss...,-1,23
16238,opinião O termo empreendedorismo nada mais é d...,0,62


In [ ]:
df_copy = df.copy()
df_copy

,text,likes_faixa
0,Tremores de terra sacudiram Goma na República ...,-1
1,O novo presidente do Equador Guillermo Lasso f...,-1
2,Novos dados sobre pesquisadores de Wuhan aumen...,0
3,RT A Ericsson disse que pode obter uma fatia ...,-1
4,Aeroportos devem ser foco para conter a varian...,0
...,...,...
16235,Este conteúdo faz parte da NexoEDUnoEnem noss...,-1
16236,AcessoLivre Ao longo do mês de dezembro o Ne...,-1
16237,Este conteúdo faz parte da NexoEDUnoEnem noss...,-1
16238,opinião O termo empreendedorismo nada mais é d...,0


## **2.1 Reconhecimento dos dados (Drive)**

In [ ]:
df_copy.columns                            # Informação de quais são as colunas do Data Frame, e também, como estão nomeadas

Index(['text', 'likes_faixa'], dtype='object')

In [ ]:
df_copy['text'][16238]

'opinião O termo empreendedorismo nada mais é do que a expressão invertida da eliminação de qualquer rede de prot… '

## **3.2 Testes utiliziando bibliotecas**


### **3.2.1 CountVectorizer**

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

### Universo de palavras - corpus

corpus      =  np.array(df_final['text'])                   # Gerando uma sequência de textos(conteúdo dos Tweets). Isso é necessário para o Vectorizer

### Criação da funcionalidade

vectorizer_cv   =   CountVectorizer()                  
countvec        =   vectorizer_cv.fit_transform(corpus.astype('U'))  # Numera quantas vezes uma palavra aprendida (após analisar toda a coluna) aparece em uma instância (mensagem)
array_vec       =   countvec.toarray()                      # Matriz de arrays com número de aparições de uma palavra aprendida em uma determinada mensagem
words_lrnd      =   vectorizer_cv.get_feature_names()       # Palavras aprendidas com os dados 

print(words_lrnd[9532:9540])                                # Amostra de algumas palavras aprendidas

['favoravelmente', 'favorece', 'favorecer', 'favoreceram', 'favorecerem', 'favoreceria', 'favoreça', 'favoritas']


In [ ]:
### Amostra do conteúdo passado como texto.
### Nota-se que junto aos textos são passados links, possivelmente e-mails.
df_copy['text'][0]

'Tremores de terra sacudiram Goma na República Democrática do Congo nesta segundafeira  assustando moradores… '

In [ ]:
print(vectorizer_cv.vocabulary_)       # Apresenta um dicionário com as palavras e o numero delas dentro do vocabulário 

print("O número de aparições da palavra 'tremores' dentro dessa mensagem é: {}".format(array_vec[0][22035]))

{'tremores': 22035, 'de': 5990, 'terra': 21490, 'sacudiram': 19902, 'goma': 10681, 'na': 15065, 'república': 19236, 'democrática': 6235, 'do': 7323, 'congo': 4935, 'nesta': 15293, 'segundafeira': 20195, 'assustando': 1908, 'moradores': 14752, 'novo': 15504, 'presidente': 17519, 'equador': 8301, 'guillermo': 10913, 'lasso': 13055, 'foi': 9962, 'empossado': 7906, 'em': 7806, 'capital': 3545, 'quito': 18389, 'cerimô': 3911, 'novos': 15505, 'dados': 5907, 'sobre': 20621, 'pesquisadores': 16801, 'wuhan': 23251, 'aumentam': 2148, 'debate': 5995, 'origens': 15945, 'da': 5900, 'covid': 5541, 'rt': 19804, 'ericsson': 8335, 'disse': 7203, 'que': 18298, 'pode': 17055, 'obter': 15627, 'uma': 22321, 'fatia': 9509, 'menor': 14289, 'no': 15381, 'lançamento': 13026, 'china': 4057, 'sua': 20877, 'participação': 16315, 'mercado': 14325, 'infra': 11953, 'aeroportos': 489, 'devem': 6894, 'ser': 20341, 'foco': 9948, 'para': 16192, 'conter': 5155, 'variante': 22570, 'índia': 23433, 'diz': 7303, 'infectologi

### **3.2.2 TfidfVectorizer**


https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


In [ ]:
### TfidfVectorizer é melhor para datasets maiores(muito grandes), além disso possui mais features para stop_words, lowercase, frequency e etc.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tf   =   TfidfVectorizer(min_df=1)
tfidfvec        =   vectorizer_tf.fit_transform(corpus.astype('U'))   # Contabiliza a frequencia de uma palavra dentro de uma mensagem em relação a quantas aparições no total ela tem
array_vec       =   tfidfvec.toarray()                    # Matriz de arrays com as frequências de uma palavra aprendida em uma determinada mensagem
words_lrnd2     =   vectorizer_tf.get_feature_names()     # Palavras aprendidas com os dados 

print(words_lrnd2[9532:9540])        

['favoravelmente', 'favorece', 'favorecer', 'favoreceram', 'favorecerem', 'favoreceria', 'favoreça', 'favoritas']


In [ ]:
print(vectorizer_tf.vocabulary_)       # Apresenta um dicionário com as palavras e o numero delas dentro do vocabulário 

print("A frequência de aparições da palavra 'tremores' dentro dessa mensagem é: {}%".format(round(100*(array_vec[0][22035]),2)))

{'tremores': 22035, 'de': 5990, 'terra': 21490, 'sacudiram': 19902, 'goma': 10681, 'na': 15065, 'república': 19236, 'democrática': 6235, 'do': 7323, 'congo': 4935, 'nesta': 15293, 'segundafeira': 20195, 'assustando': 1908, 'moradores': 14752, 'novo': 15504, 'presidente': 17519, 'equador': 8301, 'guillermo': 10913, 'lasso': 13055, 'foi': 9962, 'empossado': 7906, 'em': 7806, 'capital': 3545, 'quito': 18389, 'cerimô': 3911, 'novos': 15505, 'dados': 5907, 'sobre': 20621, 'pesquisadores': 16801, 'wuhan': 23251, 'aumentam': 2148, 'debate': 5995, 'origens': 15945, 'da': 5900, 'covid': 5541, 'rt': 19804, 'ericsson': 8335, 'disse': 7203, 'que': 18298, 'pode': 17055, 'obter': 15627, 'uma': 22321, 'fatia': 9509, 'menor': 14289, 'no': 15381, 'lançamento': 13026, 'china': 4057, 'sua': 20877, 'participação': 16315, 'mercado': 14325, 'infra': 11953, 'aeroportos': 489, 'devem': 6894, 'ser': 20341, 'foco': 9948, 'para': 16192, 'conter': 5155, 'variante': 22570, 'índia': 23433, 'diz': 7303, 'infectologi

## **3.3 Aplicação definitiva das bibliotecas nos dados**

In [ ]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
language = "portuguese"
stopwords_list  = stopwords.words(language)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
### TfidfVectorizer é melhor para datasets maiores(muito grandes), além disso possui mais features para stop_words, lowercase, frequency e etc.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tf   =   TfidfVectorizer(min_df = 2, max_df = 0.5, stop_words = stopwords_list)
tfidfvec        =   vectorizer_tf.fit_transform(corpus.astype('U'))   # Contabiliza a frequencia de uma palavra dentro de uma mensagem em relação a quantas aparições no total ela tem
array_vec       =   tfidfvec.toarray()                    # Matriz de arrays com as frequências de uma palavra aprendida em uma determinada mensagem
words_lrnd2     =   vectorizer_tf.get_feature_names()     # Palavras aprendidas com os dados 

print("\n",vectorizer_tf.vocabulary_)                          # Apresenta um dicionário com as palavras e o numero delas dentro do vocabulário 
print("\n",words_lrnd2[9532:9540])


 {'tremores': 11209, 'terra': 10930, 'república': 9820, 'democrática': 3184, 'congo': 2515, 'nesta': 7783, 'segundafeira': 10305, 'moradores': 7494, 'novo': 7882, 'presidente': 8929, 'equador': 4217, 'guillermo': 5557, 'lasso': 6611, 'empossado': 4029, 'capital': 1796, 'quito': 9417, 'cerimô': 1969, 'novos': 7883, 'dados': 3023, 'sobre': 10526, 'pesquisadores': 8538, 'wuhan': 11856, 'aumentam': 1118, 'debate': 3061, 'origens': 8107, 'covid': 2832, 'rt': 10094, 'ericsson': 4235, 'disse': 3661, 'pode': 8662, 'obter': 7936, 'fatia': 4833, 'menor': 7258, 'lançamento': 6594, 'china': 2058, 'participação': 8295, 'mercado': 7279, 'aeroportos': 257, 'devem': 3486, 'ser': 10391, 'foco': 5073, 'conter': 2646, 'variante': 11499, 'índia': 11935, 'diz': 3721, 'infectologista': 6046, 'voltado': 11756, 'pequenos': 8444, 'negócios': 7765, 'microcrédito': 7325, 'tipo': 10999, 'empréstimo': 4051, 'oferece': 7980, 'vantagens': 11487, 'outros': 8132, 'têm': 11324, 'creditas': 2852, 'resolveu': 9847, 'faz

## **4.2 Separando os dados para treino e teste**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_copy['text'],df_copy['likes_faixa'], test_size = 0.25)

In [ ]:
X_train

3482     "Essa estratégia  que é usada pelos bolsonaris...
3156     Sindicatos colombianos estão se preparando par...
6105     "No esquema revelado pelo Estadão contudo quem...
11773    RT  perceba bolsonaro e seus súditos mudaram o...
8830     Remédio de cannabis é autorizado na Anvisa mas...
                               ...                        
4954     Conteúdo patrocinado por  O governador Mauro M...
535      Especialistas alerta para terceira onda da Cov...
2908     Dois estudantes e um adulto ficaram feridos em...
3749     Gino da dupla com Geno é internado com covid E...
4864     Quiz para fãs de Harry Potter marca  anos de p...
Name: text, Length: 12180, dtype: object

# **. Aplicando algoritimos de Machine Learning nos dados**

In [ ]:
### TfidfVectorizer é melhor para datasets maiores(muito grandes), além disso possui mais features para stop_words, lowercase, frequency e etc.
from sklearn.feature_extraction.text import TfidfVectorizer
corpus_final    =   np.array(X_train)
vector_final    =   TfidfVectorizer(min_df = 2, max_df = 0.5, stop_words = stopwords_list)
tfidfvec        =   vector_final.fit_transform(corpus_final.astype('U'))   # Contabiliza a frequencia de uma palavra dentro de uma mensagem em relação a quantas aparições no total ela tem
array_vec       =   tfidfvec.toarray()                    # Matriz de arrays com as frequências de uma palavra aprendida em uma determinada mensagem
words_lrnd2     =   vector_final.get_feature_names()     # Palavras aprendidas com os dados 

print("\n",vector_final.vocabulary_)                          # Apresenta um dicionário com as palavras e o numero delas dentro do vocabulário 
print("\n",words_lrnd2[9532:9540])


 {'estratégia': 3675, 'usada': 9462, 'bolsonaristas': 1223, 'desde': 2730, 'presidente': 7432, 'tomou': 9170, 'posse': 7285, 'responsável': 8221, 'sindicatos': 8728, 'colombianos': 1841, 'preparando': 7404, 'protestos': 7640, 'massa': 5899, 'redor': 8012, 'país': 6962, 'marcar': 5850, 'oitavo': 6630, 'dia': 2882, 'man': 5790, 'esquema': 3617, 'revelado': 8285, 'estadão': 3639, 'contudo': 2220, 'vota': 9787, 'governo': 4515, 'ganha': 4375, 'chance': 1638, 'projetos': 7582, 'cujo': 2429, 'rt': 8400, 'bolsonaro': 1224, 'discurso': 2975, 'passaram': 6915, 'usar': 9468, 'máscara': 6316, 'falar': 3947, 'favor': 4011, 'vacina': 9496, 'ontem': 6667, 'pra': 7324, 'remédio': 8124, 'cannabis': 1440, 'autorizado': 942, 'anvisa': 551, 'ministério': 6104, 'libera': 5554, 'sus': 8918, 'segundo': 8576, 'deputado': 2665, 'projeto': 7581, 'quer': 7807, 'erros': 3495, 'cobertura': 1796, 'código': 2475, 'penal': 7002, 'assim': 755, 'filme': 4128, 'netflix': 6445, 'baseado': 1066, 'livro': 5646, 'traz': 9

In [ ]:
### criar um classificador - MultinomialNB - Naive Bayes - teorema de Bayes - probabildiade condicional
### https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
### 3  agrupamentos -1 --  0 -- 1

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics                                 # Biblioteca para medir a acurácia do modelo

### Classificador

clf   =  MultinomialNB(alpha = 0.1)                         # Alpha = 1 (Suavização de Laplace) 

### Treinamento

clf.fit(vector_final, y_train)

ValueError: ignored

In [ ]:
vectors_test = vectorizer.transform(X_test)                # Transforma o dado de teste para o mesmo formato do dado de treino

vector_pred  = clf.predict(vectors_test)                   # Predição do modelo

In [ ]:
acc_score = metrics.accuracy_score(y_test,vector_pred)     # Teste de acurácia

In [ ]:
acc_score                                                  # 76,2608% de assertividade 

0.7626076260762608

In [ ]:
### Verificando o desempenho

vectors_test2 = vectorizer.fit_transform(X_test)

clf2   =  MultinomialNB(alpha = 0.1) 
clf2.fit(vectors_test2, y_test)

vector_pred2  = clf2.predict(vectors_test2)

In [ ]:
acc_score = metrics.accuracy_score(y_test,vector_pred2) 
acc_score

0.971709717097171

# **6. Avaliando a acurácia do(s) modelo(s)**